In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
import numpy as np

In [25]:
# URL of different Tennessee Districts
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2022&id=NH01&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=IL05&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2022&id=AL01&spec=N'


#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'
    #'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN03&spec=N'
 #'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN04&spec=N'
# 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN05&spec=N'
#  'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N'   
#  'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N'
#  'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2022&id=AL01&spec=N'

response = requests.get(URL)
soup = BS(response.content, 'html.parser')

In [29]:
#Creating empty dataframe with column names
Election = pd.DataFrame(columns = ['Name',
                                   'Party',
                                   'State', 
                                   'District_Number',
                                   'Incumbent',
                                   'Result',
                                   'Vote_Percentage',
                                   'Amount_Raised ($)',
                                   'Amount_Spent ($)'])
Election

# List of variables 
Candidate_Name = ""
Candidate_Party = ""
Candidate_State =""
Candidate_District =""
C_Incumbent = ""
Won_Race =""
Vote_Percentage = np.NaN
Amount_Raised = np.NaN
Amount_Spent = np.NaN

# Loop to find
for amount in soup.find_all("div", class_= "Members--list-item"):
    #print(amount)
    
    for member in amount.find_all("div", class_= "Members--bio u-richtext"):
        Candidate = member.find('strong').text
        #print(Candidate)
        Candidate_Party = re.findall("\(\w\)", Candidate)[0].replace('(', '').replace(')', '') 
        Candidate_Name = Candidate.split(' (')[0].replace("\t", "").replace("\n", "")
        #print(Candidate_Name)
        Candidate_Party = re.findall("\(\w\)", Candidate)[0].replace('(', '').replace(')', '') 
        #print(Candidate_Party)
        #Vote= re.findall("(\d+.\d%)", Candidate)[0].replace('(', '').replace('%',' ').replace(')', '') 
        #print(Vote)
        
        if "Incumbent" in Candidate:
            C_Incumbent = True
            #print(C_Incumbent)
        else: 
            C_Incumbent = False
            
        if "Winner" in Candidate:
            Won_Race = True
            #print(Won_Race)
        else:
            Won_Race = False
        
        if "vote" in Candidate:
            Vote= re.findall("(\d+.\d%)", Candidate)[0].replace('(', '').replace('%',' ').replace(')', '') 
            #print(Vote)
        else: 
            Vote = ' '
        
        for title in soup.find_all('h1', class_= "Hero-title"):
            #print(title)
            Candidate_State = title.text.split(' District')[0]
            #print(Candidate_State)
            Candidate_District = title.text.split('District ')[1].split(" ")[0]
            #print(Candidate_District) 
                    
            raised = amount.find_all("td", class_= "Members--number")[0].text.replace('$',"").replace(",", "")
            #print(raised)
            spent = amount.find_all("td", class_= "Members--number")[1].text.replace('$',"").replace(",", "")
            #print(spent)
            Election = Election.append({'Name': Candidate_Name, 'Party': Candidate_Party, 'State' : Candidate_State, 'District_Number': Candidate_District ,'Incumbent': C_Incumbent, 'Result': Won_Race, 'Vote_Percentage': Vote, 'Amount_Raised ($)': raised, 'Amount_Spent ($)': spent}, ignore_index = True)  
         

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_65875/2125942849.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Election = Election.append({'Name': Candidate_Name, 'Party': Candidate_Party, 'State' : Candidate_State, 'District_Number': Candidate_District ,'Incumbent': C_Incumbent, 'Result': Won_Race, 'Vote_Percentage': Vote, 'Amount_Raised ($)': raised, 'Amount_Spent ($)': spent}, ignore_index = True)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_65875/2125942849.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Election = Election.append({'Name': Candidate_Name, 'Party': Candidate_Party, 'State' : Candidate_State, 'District_Number': Candidate_District ,'Incumbent': C_Incumbent, 'Result': Won_Race, 'Vote_Percentage': Vote, 'Amount_Raised ($)': raised, 'Amount_Spent ($)': spent},

In [30]:
Election

,Name,Party,State,District_Number,Incumbent,Result,Vote_Percentage,Amount_Raised ($),Amount_Spent ($)
0,Mark Green,R,Tennessee,07,True,True,69.9,1194960,935487
1,Kiran Sreepada,D,Tennessee,07,False,False,27.3,206644,207191
2,Ronald Brown,I,Tennessee,07,False,False,2.2,1750,0
3,Scott Vieira Jr,I,Tennessee,07,False,False,0.6,655,1049
